# Symbolic derivation and linearization — Double Pendulum on a Cart

This notebook performs a symbolic derivation of the equations of motion for a **double pendulum on a cart** using `sympy`.  
It:

- Defines parameters and generalized coordinates,
- Computes positions and velocities of centers of mass,
- Constructs kinetic energy $T$ and potential energy $V$,
- Builds the Lagrangian $(\mathcal{L}=T-V)$,
- Derives the Euler–Lagrange equations and rearranges them into the manipulator form
  $$
  M(q)\ddot q + C(q,\dot q)\dot q + G(q) = B u,
  $$
- Forms the first-order state ODE $\dot x = f(x,u)$,
- Computes symbolic Jacobians $A(x,u)=\partial f/\partial x$ and $B(x,u)=\partial f/\partial u$,
- Derives linearizition at the upright equilibrium.


In [1]:
# Cell: Imports and symbols
import sympy as sp
import numpy as np
import pickle
from sympy import sin, cos, simplify

# Symbolic variables for generalized coords and their derivatives
x, th1, th2 = sp.symbols('x th1 th2')                    # positions
dx, dth1, dth2 = sp.symbols('dx dth1 dth2')              # velocities
ddx, ddth1, ddth2 = sp.symbols('ddx ddth1 ddth2')        # accelerations

# Parameters
M, m1, m2 = sp.symbols('M m1 m2')
l1, lc1, lc2 = sp.symbols('l1 lc1 lc2')                  # lengths: l1 is joint distance, lc* are COM distances
I1, I2 = sp.symbols('I1 I2')                             # rotational inertias about COM
g = sp.symbols('g')                                      # gravity

# Input
u = sp.symbols('u')

# Pack vectors
q = sp.Matrix([x, th1, th2])
dq = sp.Matrix([dx, dth1, dth2])
ddq = sp.Matrix([ddx, ddth1, ddth2])


In [2]:
# Cell: Positions of centers of mass and Jacobians
# COM positions (absolute angles from vertical)
x1 = x + lc1*sp.sin(th1)
y1 = -lc1*sp.cos(th1)

x2 = x + l1*sp.sin(th1) + lc2*sp.sin(th2)
y2 = -l1*sp.cos(th1) - lc2*sp.cos(th2)

r1 = sp.Matrix([x1, y1])
r2 = sp.Matrix([x2, y2])

# Generalized coordinate vector for Jacobian usage
q_syms = sp.Matrix([x, th1, th2])
dq_syms = sp.Matrix([dx, dth1, dth2])

# Jacobians: velocity = J * dq
Jr1 = r1.jacobian(q_syms)
Jr2 = r2.jacobian(q_syms)

v1 = Jr1 * dq_syms
v2 = Jr2 * dq_syms

# Squared speeds (simplified)
v1_sq = simplify(v1.dot(v1))
v2_sq = simplify(v2.dot(v2))

v1_sq, v2_sq  # show expressions


(dth1**2*lc1**2 + 2*dth1*dx*lc1*cos(th1) + dx**2,
 dth1**2*l1**2 + 2*dth1*dth2*l1*lc2*cos(th1 - th2) + 2*dth1*dx*l1*cos(th1) + dth2**2*lc2**2 + 2*dth2*dx*lc2*cos(th2) + dx**2)

In [3]:
# Cell: Kinetic and potential energy and Lagrangian
T_cart = sp.Rational(1,2) * M * dx**2
T1 = sp.Rational(1,2) * m1 * v1_sq + sp.Rational(1,2) * I1 * dth1**2
T2 = sp.Rational(1,2) * m2 * v2_sq + sp.Rational(1,2) * I2 * dth2**2
T = simplify(T_cart + T1 + T2)

# Potential energy: y negative downward, so height = -y
V1 = m1 * g * (-y1)
V2 = m2 * g * (-y2)
V = simplify(V1 + V2)

L = simplify(T - V)



In [4]:
# --- Compute derivatives ---
dL_dq  = sp.Matrix([sp.diff(L, q) for q in q_syms])
dL_ddq = sp.Matrix([sp.diff(L, dq) for dq in dq_syms])

# Time derivative of dL_ddq using chain rule
d_dt_dL_ddq = sp.Matrix([
    sum(sp.diff(dL_ddq[i], q_syms[j]) * dq_syms[j] for j in range(3)) +
    sum(sp.diff(dL_ddq[i], dq_syms[j]) * ddq[j] for j in range(3))
    for i in range(3)
])

# Euler–Lagrange equations
EOM = sp.simplify(d_dt_dL_ddq - dL_dq)
EOM


Matrix([
[                                                        ddth1*(l1*m2 + lc1*m1)*cos(th1) + ddth2*lc2*m2*cos(th2) + ddx*(M + m1 + m2) - dth1**2*(l1*m2 + lc1*m1)*sin(th1) - dth2**2*lc2*m2*sin(th2)],
[I1*ddth1 + ddth1*l1**2*m2 + ddth1*lc1**2*m1 + ddth2*l1*lc2*m2*cos(th1 - th2) + ddx*l1*m2*cos(th1) + ddx*lc1*m1*cos(th1) + dth2**2*l1*lc2*m2*sin(th1 - th2) - g*l1*m2*sin(th1) - g*lc1*m1*sin(th1)],
[                                                             ddth1*l1*lc2*m2*cos(th1 - th2) + ddth2*(I2 + lc2**2*m2) + ddx*lc2*m2*cos(th2) - dth1**2*l1*lc2*m2*sin(th1 - th2) - g*lc2*m2*sin(th2)]])

In [5]:
# Extract M and H 

ddq_syms = [ddx, ddth1, ddth2]

# Build M matrix by extracting coefficients of accelerations
M_mat = sp.zeros(3, 3)
H_vec  = sp.zeros(3, 1)
for i in range(3):
    expr = sp.expand(EOM[i])         # EOM[i] symbolic expression
    for j, dd_sym in enumerate(ddq_syms):
        coeff = sp.simplify(sp.diff(expr, dd_sym))
        M_mat[i, j] = coeff
    # remainder after subtracting M*ddq terms
    H_vec[i] = sp.simplify(expr - sum(M_mat[i, j] * ddq_syms[j] for j in range(3)))

# Optional: simplify
M_mat = sp.simplify(M_mat)
H_vec  = sp.simplify(H_vec)

# Quick display
M_mat, H_vec


(Matrix([
 [              M + m1 + m2, (l1*m2 + lc1*m1)*cos(th1),          lc2*m2*cos(th2)],
 [(l1*m2 + lc1*m1)*cos(th1), I1 + l1**2*m2 + lc1**2*m1, l1*lc2*m2*cos(th1 - th2)],
 [          lc2*m2*cos(th2),  l1*lc2*m2*cos(th1 - th2),           I2 + lc2**2*m2]]),
 Matrix([
 [-dth1**2*l1*m2*sin(th1) - dth1**2*lc1*m1*sin(th1) - dth2**2*lc2*m2*sin(th2)],
 [    dth2**2*l1*lc2*m2*sin(th1 - th2) - g*l1*m2*sin(th1) - g*lc1*m1*sin(th1)],
 [                           -lc2*m2*(dth1**2*l1*sin(th1 - th2) + g*sin(th2))]]))